 ## Security and advanced SQL

``<jeep@cphbusiness.dk>``

## Agenda

* Security in PostgreSQL
  * Users and user-access
  * Row-level security
  * `pg_hba.conf`
  * Backups
* Advanced read/write statistics
* Exotic uses of SQL databases
  * Prepared statements
  * Continuous views
  * Cursors
  * Foreign tables
* Course learning outcome
* Assignment 10

# Security

(Our last serious topic)

* Users
* Row-level security
* Access configuration
* Backup strategies

## Users in SQL

* RDBMS are managed by users (roles) with certain permissions
  * `CREATE USER name`

* `SUPERUSER`: Permission to override access restrictions
* `CREATEDB`: Permission to create databases
* `CONNECTION LIMIT`: Amount of connections

* Just like other SQL entities users can be `ALTER`ed or `DROP`ped

## The `GRANT` command

* Permissions can be given on a very granular level
  * `GRANT [SELECT | INSERT | UPDATE ...] ON table TO name`
  
* Example: `GRANT INSERT ON mytable TO myuser`

* Permissions can also be `REVOKE`d

* Example: `REVOKE INSERT ON mytable FROM myuser`

## User creation example

In [149]:
%load_ext sql
%sql postgres://appdev@0.0.0.0/appdev

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: appdev@appdev'

In [150]:
%sql CREATE USER myuser;

Done.


[]

In [157]:
%sql postgres://myuser@0.0.0.0/appdev

'Connected: myuser@appdev'

In [158]:
%sql INSERT INTO hello(language, hello) VALUES('Yiddish', 'אַ גוט יאָר');

1 rows affected.


[]

In [156]:
%sql GRANT SELECT, INSERT ON hello TO myuser

Done.


[]

## User exercise

During the exercise you might encounter problems. Your job is to fix them so you can get the job done

* Create a user with a cool name and login with that user
* Login with that user and create a database with another cool name
* Login with the `appdev` user and 
  * Create a table within your new database
  * Add a column (`language`) that references the `language` column from the `hello` table within the `appdev` database
* Login with your new user and insert the equivalent of "hello world" in the `whitespace` programming language

## User access

* `pg_hba.conf` is a configuration file for allowed logins
  * So far: login with password

* Best practice: Only allow localhost logins
  * Login with SSH tunnel

* Unfortunately this requires ssh to be installed on the PostgreSQL server (the docker image)

* See https://www.postgresql.org/docs/current/static/ssh-tunnels.html

# Row-level security

* Permissions can be set on table- or even row level
  * `CREATE POLICY name ON table TO role [USING expression] [WITH CHECK expression]`
  * `USING` is checked during row selection
  * `WITH CHECK` is checked during `INSERT` and `UPDATE`

* Must be enabled: `ALTER TABLE table ENABLE ROW LEVEL SECURITY`

* Quick note: can cause problems in select statements, see https://www.dbrnd.com/2016/08/postgresql-9-5-using-for-update-skip-locked-option-select-only-committed-records-mvcc-lock-with-nowait-thread/

In [160]:
%sql postgres://appdev@0.0.0.0/appdev
%sql CREATE POLICY my_policy ON hello TO myuser USING (language = 'Yiddish')

Done.


[]

In [162]:
%sql postgres://myuser@0.0.0.0/appdev
%sql SELECT * FROM hello;

55 rows affected.


language,hello
Amharic (አማርኛ),ሠላም
Arabic (العربيّة),السّلام عليكم
Armenian (հայերեն),Բարև ձեզ
Bengali (বাংলা),নমস্কার
Braille,⠓⠑⠇⠇⠕
Burmese (မြန်မာ),မင်္ဂလာပါ
C,"printf (Hello, world!\n);"
Czech (čeština),Dobrý den
Danish (dansk),Hej / Goddag / Halløj
Dutch (Nederlands),Hallo / Dag


In [ ]:
%sql ALTER TABLE hello ENABLE ROW LEVEL SECURITY

## SSH tunnel example

`pg_hba.conf`    
    # TYPE  DATABASE        USER            ADDRESS                 METHOD
    host    all             all             127.0.0.1/32            trust

* `ssh -L 63333:localhost:5432 joe@foo.com`

* `psql -h localhost -p 63333 postgres`

## Backups

* Backups are important - even if you are on a cluster!!
  * See https://www.postgresql.org/docs/current/static/backup.html

* Backing up: `pg_dump database`
  * Example: `pg_dump appdev > appdev.sql`

* Restoring from dump: `psql < dumpfile`
  * Example: `psql -U appdev < dumpfile`

## Backup exercise

See https://www.postgresql.org/docs/current/static/backup.html

* Restart your docker container to kill all connections
* Backup your appdev database using `pg_dump` from inside your container
* Destroy the `appdev` database by
  1. Connect with your *appdev* database with your *appdev* user `psql appdev appdev`
  2. Change to another database (for instance the postgres database) by writing `\c postgres`
    * You can list databases with `\l`
  3. Run `DROP DATABASE appdev;`
* Realise that all your data is gone
* Restore the database by
  1. Re-create the database by logging in with the postgres user and creating the database
  2. `GRANT` the `appdev` user permission to do whatever it want on the database
  3. Exit the database and restore the data using `psql -U appdev`

# Advanced statistics

* Tons of statistics available in the `pg_catalog` schema
  * See https://www.postgresql.org/docs/current/static/monitoring-stats.html

* `pg_stat_activity`
* `pg_stat_database`
* `pg_stat_all_indexes`
* `pg_stat_all_tables`

## Statistics exercise

Source: https://www.postgresql.org/docs/current/static/monitoring-stats.html

* Figure out how many connections you have open right now
* Find the table where you've scanned the most tuples sequentially
* Find the index where you've fetched the most tuples from

# Exotic uses of SQL databases

* Prepared statements
* Cursors
* Continuous views
* Foreign tables

## Prepared statements

* Prepared statements can be hard-coded into the database
  * See https://www.postgresql.org/docs/current/static/sql-prepare.html
  
  
    PREPARE fooplan (int, text, bool, numeric) AS
    INSERT INTO foo VALUES($1, $2, $3, $4);
    EXECUTE fooplan(1, 'Hunter Valley', 't', 200.00);

## Cursors

* A query can be broken up into piece with cursors
  * Similar to a text cursor pointer or file iterator
* Useful for accumulating a specific result before returning

* Needs to be `DECLARE`d, and then you can start to `FETCH` data

    BEGIN WORK;

    -- Set up a cursor:
    DECLARE liahona SCROLL CURSOR FOR SELECT * FROM films;

    -- Fetch the first 5 rows in the cursor liahona:
    FETCH FORWARD 5 FROM liahona;

     code  |          title          | did | date_prod  |   kind   |  len
    -------+-------------------------+-----+------------+----------+-------
     BL101 | The Third Man           | 101 | 1949-12-23 | Drama    | 01:44
     BL102 | The African Queen       | 101 | 1951-08-11 | Romantic | 01:43
     JL201 | Une Femme est une Femme | 102 | 1961-03-12 | Romantic | 01:25
     P_301 | Vertigo                 | 103 | 1958-11-14 | Action   | 02:08
     P_302 | Becket                  | 103 | 1964-02-03 | Drama    | 02:28

    -- Fetch the previous row:
    FETCH PRIOR FROM liahona;

     code  |  title  | did | date_prod  |  kind  |  len
    -------+---------+-----+------------+--------+-------
     P_301 | Vertigo | 103 | 1958-11-14 | Action | 02:08

    -- Close the cursor and end the transaction:
    CLOSE liahona;
    COMMIT WORK;

## Continuous views

An idea from PipelineDB: https://www.pipelinedb.com/

Imagine real-time analytics and data flows. Not very handy with modern RDBMS. Or is it?

    CREATE CONTINUOUS VIEW latency AS
    SELECT
      percentile_cont(array[90, 95, 99])
    WITHIN GROUP (ORDER BY latency::integer)
    FROM latency_stream; 

# SQL part: learning outcome

## Knowledge
The student must have knowledge of:

 * Various database types and the underlying models
 * A specific database system’s storage organisation  and query execution
 * A specific database system’s optimisation possibilities – including advantages and disadvantages
 * Database-specific security problems and their solutions
 * Concepts and issues when handling big data
 * The particular issues raised by having many simultaneous transactions, including in connection with distributed databases
 * Relational algebra (including its relationship to execution plans)

## Skills
The student can:

 * Transform logical data models into physical models in various database types
 * Implement database optimisation
 * Use parts of the administration tool to assist in the optimisation and tuning of existing databases, including the incorporation of a specific DBMS’ execution plans
 * Use a specific database system’s tools for handling simultaneous transactions
 * Use the programming and other facilities provided by a modern DBMS

# Assignment 10: Modelling OECD data

* **Deadline**: 11th of May 12:00
* **Review deadline**: 13th of May 23:59

This assignment aims to wrap up all that you've learned so far. Your job is to go from some data, design a logical data model, transfer it to a physical data model, insert the data and finally get some relevant information from it.

The domain is extracted from OECD data (https://data.oecd.org/) and we'll be looking at Gross Domestic Product (GDP), education, and life expectancy over time. The data is available via Peergrade in the files `gdp.csv`, `education.csv` and `lifeexpectancy.csv` respectively. Education is defined as the percentage of people in a country that is below upper secondary school (lower values means a more well-educated population). 

1. Have a look at the files and draw a logical data model
2. Transfer that logical data model into a physical data model and create the necessary tables in PostgreSQL
3. Load all the data into your database
  * Hint: You can do this in Python very easily if you load the csv data using Pandas (`pandas.read_csv`) and iterate over the rows
4. Find the country whose GDP has grown the most over time and plot the level of education on the x axis and the life expectancy on the y axis. Do the same thing for the country whose GDP has grown the least over time

Hand-in: A drawing of your logical data model. The SQL you used to create your physical model and the two plots.